## Imports

In [ ]:
import pandas as pd
import sys
sys.path.append('../')
from src.trading_env_final import TradingEnv

from stable_baselines3 import A2C
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv

## Create Env

In [ ]:
df = pd.read_csv("../data/PNB_Indicators.csv")
df_option_data = pd.read_csv("../data/PNB_Options_Data.csv")

options_path = "../data/CSV" # has date realted to options on each day

window_size = 10
start_index = 748 # 2015-01-01
end_index = 2723 # 2022-12-30
frame_bound = (start_index, end_index)
margin = 1000000000
lot_size = 1000
spread = 5 # no.of stikes from atm to consider as itm and otm

env = TradingEnv(df, df_option_data, window_size, frame_bound, margin, lot_size, spread, options_path)

In [ ]:
t,t1 = env.reset()

In [ ]:
check_env(env)
t,t1 = env.reset()

## Train Env

In [ ]:
env = DummyVecEnv([lambda: env])

In [ ]:
# This values are to be filled after hyperparameter tuning if not just run withuot passing these values
learning_rate = 0.0005367590641575743
gamma = 0.9315531162894451
gae_lambda = 0.9210798936175724
n_steps = 5
entropy_coef = 0.049835574208271116

In [ ]:
model = A2C("MlpPolicy", env, verbose=1,
                learning_rate=learning_rate,
                gamma=gamma,
                gae_lambda=gae_lambda,
                n_steps=n_steps,
                ent_coef=entropy_coef)

In [ ]:
model.learn(total_timesteps=100000) # 98750 for 50 episodes

In [ ]:
model_path = "../Model/a2c_trading_env2_spread5_final"
model.save(model_path)

# BackTesting

In [ ]:
options_path1 = "../data/CSV" 
frame_bound=(2725, 3040) #2023-01-02 to 2024-04-10
speread = 5
new_env = TradingEnv(df, df_option_data, window_size, frame_bound, margin, lot_size, spread, options_path)
new_env = DummyVecEnv([lambda: new_env])

In [ ]:
model_path = "../Model/a2c_trading_env2_spread5_final"
loaded_model = A2C.load(model_path)                    

In [ ]:
obs = new_env.reset()
done = False
while not done:
    action, _states = loaded_model.predict(obs)
    obs, rewards, done, info = new_env.step(action)

In [ ]:
print(info[0]['total_profit'])
print(info[0]['total_reward'])
print(info[0]['drawdown'])